## Mississippi Valley Forest Birds - Protection

This indicator was used in the 2023 base blueprint. This code creates the full extent raster.

Created by Amy Keister, last run by Amy Keister on 1 June 2023. It took 2 minutes to run

In [66]:
import os
import arcpy

In [67]:
import time
start = time.time()

In [68]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\MississippiAlluvialValleyForestBirds_P\MAVProtect.gdb"

In [69]:
# define final indicator outputs
Out = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\MississippiAlluvialValleyForestBirds_P\MississippiAlluvialValleyForestBirds_Protection.tif"

In [70]:
# define sub-indicator outputs

In [71]:
# define rasters used for cell size, extent, and snapping
Rextent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [72]:
# define inputs
FP= r"F:\GIS_DATA\DecisionSupportTools\LMVJV\FPDSM\MAV_FPDSM_FINAL\MAV_FPDSM_v7_noCE_final_2.img"

## Start Analysis

In [73]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [74]:
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\MississippiAlluvialValleyForestBirds_P\MAVProtect.gdb


## bring in LMVJV forest protection DSM for interior forest breeding birds of the MAV

In [10]:
# make a copy of the LMVJV .img
arcpy.conversion.RasterToGeodatabase(FP, OutWorkspace, '')

<Result ''>

In [11]:
# reclassify to bin

# ArcGIS reclass includes the second value in the class for example,
# 0 1 is <=1 (1 is included)
# 1 5 is >1 - 5 (1 is not included, 5 is included)
# 5 20 is >5 - 20 (5 is not included, 20 is included)

out_raster = arcpy.sa.Reclassify("MAV_FPDSM_v7_noCE_final_2", "VALUE", "0 0 0;1 10 1;10 20 2;20 30 3;30 40 4;40 50 5;50 60 6;60 70 7;70 80 8;80 90 9;90 100 10", "DATA"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [81]:
# clip to SE 2022 Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("SEMask")

In [87]:
# export as .tif
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("SEMask", Out, '', None, "15", "NONE", "NONE", "4_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [88]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 10:
        return '10 = Highest priority forest breeding bird habitat patch for future protection (score >90-100)'
    elif value == 9:
        return '9 = (score >80-90)'
    elif value == 8:
        return '8 = (score >70-80)'
    elif value == 7:
        return '7 = (score >60-70)'
    elif value == 6:
        return '6 = (score >50-60)'
    elif value == 5:
        return '5 = (score >40-50)'
    elif value == 6:
        return '6 = (score >40-50)'
    elif value == 4:
        return '4 = (score >30-40)'
    elif value == 3:
        return '3 = (score >20-30)'
    elif value == 2:
        return '2 = (score >10-20)'
    elif value == 1: 
        return '1 = Low priority (score >0-10)'
    elif value == 0: 
        return '0 = Not a priority (score =0)'
"""

In [89]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(Out, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\MississippiAlluvialValleyForestBirds_P\\MississippiAlluvialValleyForestBirds_Protection.tif'>

In [90]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 10:
		return 51
	if Value == 9:
		return 90
	if Value == 8:
		return 126
	if Value == 7:
		return 163
	if Value == 6:
		return 201
	if Value == 5:
		return 234
	if Value == 4:
		return 251
	if Value == 3:
		return 255
	if Value == 2:
		return 255
	if Value == 1:
		return 253
	else:
		return 255
		
def Reclass2(Value):
	if Value == 10:
		return 16
	if Value == 9:
		return 22
	if Value == 8:
		return 36
	if Value == 7:
		return 48
	if Value == 6:
		return 60
	if Value == 5:
		return 80
	if Value == 4:
		return 117
	if Value == 3:
		return 160
	if Value == 2:
		return 202
	if Value == 1:
		return 254
	else:
		return 255
		
def Reclass3(Value):
	if Value == 10:
		return 104
	if Value == 9:
		return 127
	if Value == 8:
		return 130
	if Value == 7:
		return 127
	if Value == 6:
		return 114
	if Value == 5:
		return 94
	if Value == 4:
		return 84
	if Value == 3:
		return 100
	if Value == 2:
		return 130
	if Value == 1:
		return 167
	else:
		return 255
		
"""

In [91]:
# calculate Red field
arcpy.management.CalculateField(Out, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(Out, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(Out, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\MississippiAlluvialValleyForestBirds_P\\MississippiAlluvialValleyForestBirds_Protection.tif'>

In [18]:
end = time.time()
print(end - start)

263.945853471756
